# Data Processing

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline

In [2]:
# Read data from the CSV into a dataframe
raw_1 = pd.read_csv('00-01.csv')
raw_2 = pd.read_csv('01-02.csv')
raw_3 = pd.read_csv('02-03.csv')
raw_4 = pd.read_csv('03-04.csv')
raw_5 = pd.read_csv('04-05.csv')
raw_6 = pd.read_csv('05-06.csv')
raw_7 = pd.read_csv('06-07.csv')
raw_8 = pd.read_csv( '07-08.csv')
raw_9 = pd.read_csv('08-09.csv')
raw_10 = pd.read_csv('09-10.csv')
raw_11 = pd.read_csv('10-11.csv')
raw_12 = pd.read_csv('11-12.csv')
raw_13 = pd.read_csv( '12-13.csv')
raw_14 = pd.read_csv('13-14.csv')
raw_15 = pd.read_csv('14-15.csv')
raw_16 = pd.read_csv('15-16.csv')
raw_17 = pd.read_csv('16-17.csv')
raw_18 = pd.read_csv('17-18.csv')

## Sort Data By Date and Get Required Columns/Features

In [3]:
# Parse data by date 

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%y').date()
    

def parse_date_Y(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%Y').date()

raw_1.Date = raw_1.Date.apply(parse_date)    
raw_2.Date = raw_2.Date.apply(parse_date)    
raw_3.Date = raw_3.Date.apply(parse_date_Y)         # Big Y  
raw_4.Date = raw_4.Date.apply(parse_date)    
raw_5.Date = raw_5.Date.apply(parse_date)    
raw_6.Date = raw_6.Date.apply(parse_date)    
raw_7.Date = raw_7.Date.apply(parse_date)    
raw_8.Date = raw_8.Date.apply(parse_date)    
raw_9.Date = raw_9.Date.apply(parse_date)    
raw_10.Date = raw_10.Date.apply(parse_date)
raw_11.Date = raw_11.Date.apply(parse_date)
raw_12.Date = raw_12.Date.apply(parse_date)
raw_13.Date = raw_13.Date.apply(parse_date)
raw_14.Date = raw_14.Date.apply(parse_date)
raw_15.Date = raw_15.Date.apply(parse_date)
raw_16.Date = raw_16.Date.apply(parse_date)
raw_17.Date = raw_17.Date.apply(parse_date)
raw_18.Date = raw_18.Date.apply(parse_date)

#Get the required column                 
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']

play_stat_1 = raw_1[columns_req]                      
play_stat_2 = raw_2[columns_req]
play_stat_3 = raw_3[columns_req]
play_stat_4 = raw_4[columns_req]
play_stat_5 = raw_5[columns_req]
play_stat_6 = raw_6[columns_req]
play_stat_7 = raw_7[columns_req]
play_stat_8 = raw_8[columns_req]
play_stat_9 = raw_9[columns_req]
play_stat_10 = raw_10[columns_req]
play_stat_11 = raw_11[columns_req]   
play_stat_12 = raw_12[columns_req]
play_stat_13 = raw_13[columns_req]
play_stat_14 = raw_14[columns_req]
play_stat_15 = raw_15[columns_req]
play_stat_16 = raw_16[columns_req]
play_stat_17 = raw_17[columns_req]
play_stat_18 = raw_18[columns_req]

In [4]:
play_stat_1.groupby('HomeTeam').mean().T.columns


Index(['Arsenal', 'Aston Villa', 'Bradford', 'Charlton', 'Chelsea', 'Coventry',
       'Derby', 'Everton', 'Ipswich', 'Leeds', 'Leicester', 'Liverpool',
       'Man City', 'Man United', 'Middlesbrough', 'Newcastle', 'Southampton',
       'Sunderland', 'Tottenham', 'West Ham'],
      dtype='object', name='HomeTeam')

## Goal Scored and Coceded 得失球

In [5]:
# Gets the goals scored
def get_goals_scored(play_stat):
    # Team names as keys
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # list containing the match location be the value corresponding to keys
    for i in range(len(play_stat)):
        HTGS = play_stat.iloc[i]['FTHG']
        ATGS = play_stat.iloc[i]['FTAG']
        teams[play_stat.iloc[i].HomeTeam].append(HTGS)
        teams[play_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Gets the goals conceded
def get_goals_conceded(play_stat):
    # Team names as keys
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # list containing the match location be the value corresponding to keys
    for i in range(len(play_stat)):
        ATGC = play_stat.iloc[i]['FTHG']
        HTGC = play_stat.iloc[i]['FTAG']
        teams[play_stat.iloc[i].HomeTeam].append(HTGC)
        teams[play_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded


def get_gsc(play_stat):
    GC = get_goals_conceded(play_stat)
    GS = get_goals_scored(play_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        #逢十場=一個LOOP
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    play_stat['HTGS'] = HTGS
    play_stat['ATGS'] = ATGS
    play_stat['HTGC'] = HTGC
    play_stat['ATGC'] = ATGC
    
    return play_stat


# Apply to each dataset
play_stat_1 = get_gsc(play_stat_1)
play_stat_2 = get_gsc(play_stat_2)
play_stat_3 = get_gsc(play_stat_3)
play_stat_4 = get_gsc(play_stat_4)
play_stat_5 = get_gsc(play_stat_5)
play_stat_6 = get_gsc(play_stat_6)
play_stat_7 = get_gsc(play_stat_7)
play_stat_8 = get_gsc(play_stat_8)
play_stat_9 = get_gsc(play_stat_9)
play_stat_10 = get_gsc(play_stat_10)
play_stat_11 = get_gsc(play_stat_11)
play_stat_12 = get_gsc(play_stat_12)
play_stat_13 = get_gsc(play_stat_13)
play_stat_14 = get_gsc(play_stat_14)
play_stat_15 = get_gsc(play_stat_15)
play_stat_16 = get_gsc(play_stat_16)
play_stat_17 = get_gsc(play_stat_17)
play_stat_18 = get_gsc(play_stat_18)


C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

## Get the Corresponding Points 分數

In [6]:
# 3pt for win, 1pt for Draw, 0 for Lose
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
# Iterate each match
def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39): #(20-1)*2 = 38
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points

# Change into W/L/D Classification (Ordinal)
def get_matchres(play_stat):
    # Team names as keys
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # list containing the match location be the value corresponding to keys
    for i in range(len(play_stat)):
        if play_stat.iloc[i].FTR == 'H':
            teams[play_stat.iloc[i].HomeTeam].append('W')
            teams[play_stat.iloc[i].AwayTeam].append('L')
        elif play_stat.iloc[i].FTR == 'A':
            teams[play_stat.iloc[i].AwayTeam].append('W')
            teams[play_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[play_stat.iloc[i].AwayTeam].append('D')
            teams[play_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def get_agg_points(play_stat):
    matchres = get_matchres(play_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])
        #逢十場=一個LOOP
        if ((i + 1)% 10) == 0:
            j = j + 1
            
    play_stat['HTP'] = HTP
    play_stat['ATP'] = ATP
    return play_stat
    
# Apply to each dataset
play_stat_1 = get_agg_points(play_stat_1)
play_stat_2 = get_agg_points(play_stat_2)
play_stat_3 = get_agg_points(play_stat_3)
play_stat_4 = get_agg_points(play_stat_4)
play_stat_5 = get_agg_points(play_stat_5)
play_stat_6 = get_agg_points(play_stat_6)
play_stat_7 = get_agg_points(play_stat_7)
play_stat_8 = get_agg_points(play_stat_8)
play_stat_9 = get_agg_points(play_stat_9)
play_stat_10 = get_agg_points(play_stat_10)
play_stat_11 = get_agg_points(play_stat_11)
play_stat_12 = get_agg_points(play_stat_12)
play_stat_13 = get_agg_points(play_stat_13)
play_stat_14 = get_agg_points(play_stat_14)
play_stat_15 = get_agg_points(play_stat_15)
play_stat_16 = get_agg_points(play_stat_16)
play_stat_17 = get_agg_points(play_stat_17)
play_stat_18 = get_agg_points(play_stat_18)

# 3pt for win, 1pt for Draw, 0 for Lose
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
# Iterate each match
def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39): #(20-1)*2 = 38
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points

# Change into W/L/D Classification (Ordinal)
def get_matchres(play_stat):
    # Team names as keys
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # list containing the match location be the value corresponding to keys
    for i in range(len(play_stat)):
        if play_stat.iloc[i].FTR == 'H':
            teams[play_stat.iloc[i].HomeTeam].append('W')
            teams[play_stat.iloc[i].AwayTeam].append('L')
        elif play_stat.iloc[i].FTR == 'A':
            teams[play_stat.iloc[i].AwayTeam].append('W')
            teams[play_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[play_stat.iloc[i].AwayTeam].append('D')
            teams[play_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def get_agg_points(play_stat):
    matchres = get_matchres(play_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])
        #逢十場=一個LOOP
        if ((i + 1)% 10) == 0:
            j = j + 1
            
    play_stat['HTP'] = HTP
    play_stat['ATP'] = ATP
    return play_stat
    
# Apply to each dataset
play_stat_1 = get_agg_points(play_stat_1)
play_stat_2 = get_agg_points(play_stat_2)
play_stat_3 = get_agg_points(play_stat_3)
play_stat_4 = get_agg_points(play_stat_4)
play_stat_5 = get_agg_points(play_stat_5)
play_stat_6 = get_agg_points(play_stat_6)
play_stat_7 = get_agg_points(play_stat_7)
play_stat_8 = get_agg_points(play_stat_8)
play_stat_9 = get_agg_points(play_stat_9)
play_stat_10 = get_agg_points(play_stat_10)
play_stat_11 = get_agg_points(play_stat_11)
play_stat_12 = get_agg_points(play_stat_12)
play_stat_13 = get_agg_points(play_stat_13)
play_stat_14 = get_agg_points(play_stat_14)
play_stat_15 = get_agg_points(play_stat_15)
play_stat_16 = get_agg_points(play_stat_16)
play_stat_17 = get_agg_points(play_stat_17)
play_stat_18 = get_agg_points(play_stat_18)

C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-

## Get Team Form

In [7]:
def get_form(play_stat,num):
    form = get_matchres(play_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(play_stat,num):
    form = get_form(play_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    play_stat['HM' + str(num)] = h                 
    play_stat['AM' + str(num)] = a

    
    return play_stat



def add_form_df(play_stat):
    play_stat = add_form(play_stat,1)
    play_stat = add_form(play_stat,2)
    play_stat = add_form(play_stat,3)
    play_stat = add_form(play_stat,4)
    play_stat = add_form(play_stat,5)
    return play_stat    
    
# Make changes to df
play_stat_1 = add_form_df(play_stat_1)
play_stat_2 = add_form_df(play_stat_2)
play_stat_3 = add_form_df(play_stat_3)
play_stat_4 = add_form_df(play_stat_4)
play_stat_5 = add_form_df(play_stat_5)
play_stat_6 = add_form_df(play_stat_6)
play_stat_7 = add_form_df(play_stat_7)
play_stat_8 = add_form_df(play_stat_8)
play_stat_9 = add_form_df(play_stat_9)
play_stat_10 = add_form_df(play_stat_10)
play_stat_11 = add_form_df(play_stat_11)
play_stat_12 = add_form_df(play_stat_12)
play_stat_13 = add_form_df(play_stat_13)
play_stat_14 = add_form_df(play_stat_14)
play_stat_15 = add_form_df(play_stat_15)
play_stat_16 = add_form_df(play_stat_16)
play_stat_17 = add_form_df(play_stat_17)
play_stat_18 = add_form_df(play_stat_18)

C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

play_stat_1 = play_stat_1[cols]
play_stat_2 = play_stat_2[cols]
play_stat_3 = play_stat_3[cols]
play_stat_4 = play_stat_4[cols]
play_stat_5 = play_stat_5[cols]
play_stat_6 = play_stat_6[cols]
play_stat_7 = play_stat_7[cols]
play_stat_8 = play_stat_8[cols]
play_stat_9 = play_stat_9[cols]
play_stat_10 = play_stat_10[cols]
play_stat_11 = play_stat_11[cols]
play_stat_12 = play_stat_12[cols]
play_stat_13 = play_stat_13[cols]
play_stat_14 = play_stat_14[cols]
play_stat_15 = play_stat_15[cols]
play_stat_16 = play_stat_16[cols]
play_stat_17 = play_stat_17[cols]
play_stat_18 = play_stat_18[cols]



## Get Last Year's Standing

In [9]:
Standings = pd.read_csv("EPLStandings.csv")
Standings.set_index(['Team'], inplace=True)
Standings = Standings.fillna(18)


def get_last(play_stat, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    play_stat['HomeTeamLP'] = HomeTeamLP
    play_stat['AwayTeamLP'] = AwayTeamLP
    return play_stat

play_stat_1 = get_last(play_stat_1, Standings, 0)
play_stat_2 = get_last(play_stat_2, Standings, 1)
play_stat_3 = get_last(play_stat_3, Standings, 2)
play_stat_4 = get_last(play_stat_4, Standings, 3)
play_stat_5 = get_last(play_stat_5, Standings, 4)
play_stat_6 = get_last(play_stat_6, Standings, 5)
play_stat_7 = get_last(play_stat_7, Standings, 6)
play_stat_8 = get_last(play_stat_8, Standings, 7)
play_stat_9 = get_last(play_stat_9, Standings, 8)
play_stat_10 = get_last(play_stat_10, Standings, 9)
play_stat_11 = get_last(play_stat_11, Standings, 10)
play_stat_12 = get_last(play_stat_12, Standings, 11)
play_stat_13 = get_last(play_stat_13, Standings, 12)
play_stat_14 = get_last(play_stat_14, Standings, 13)
play_stat_15 = get_last(play_stat_15, Standings, 14)
play_stat_16 = get_last(play_stat_16, Standings, 15)
play_stat_17 = get_last(play_stat_17, Standings, 16)
play_stat_18 = get_last(play_stat_18, Standings, 17)

## Get MatchWeek

In [10]:
def get_mw(play_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    play_stat['MW'] = MatchWeek
    return play_stat

play_stat_1 = get_mw(play_stat_1)
play_stat_2 = get_mw(play_stat_2)
play_stat_3 = get_mw(play_stat_3)
play_stat_4 = get_mw(play_stat_4)
play_stat_5 = get_mw(play_stat_5)
play_stat_6 = get_mw(play_stat_6)
play_stat_7 = get_mw(play_stat_7)
play_stat_8 = get_mw(play_stat_8)
play_stat_9 = get_mw(play_stat_9)
play_stat_10 = get_mw(play_stat_10)
play_stat_11 = get_mw(play_stat_11)
play_stat_12 = get_mw(play_stat_12)
play_stat_13 = get_mw(play_stat_13)
play_stat_14 = get_mw(play_stat_14)
play_stat_15 = get_mw(play_stat_15)
play_stat_16 = get_mw(play_stat_16)
play_stat_17 = get_mw(play_stat_17)
play_stat_18 = get_mw(play_stat_18)

## Final Dataframe

In [11]:
play_stat = pd.concat([play_stat_1,
                          play_stat_2,
                          play_stat_3,
                          play_stat_4,
                          play_stat_5,
                          play_stat_6,
                          play_stat_7,
                          play_stat_8,
                          play_stat_9,
                          play_stat_10,
                          play_stat_11,
                          play_stat_12,
                          play_stat_13,
                          play_stat_14,
                          play_stat_15,
                          play_stat_16,
                          play_stat_17,
                          play_stat_18,], ignore_index=True)


# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

play_stat['HTFormPtsStr'] = play_stat['HM1'] + play_stat['HM2'] + play_stat['HM3'] + play_stat['HM4'] + play_stat['HM5']
play_stat['ATFormPtsStr'] = play_stat['AM1'] + play_stat['AM2'] + play_stat['AM3'] + play_stat['AM4'] + play_stat['AM5']

play_stat['HTFormPts'] = play_stat['HTFormPtsStr'].apply(get_form_points)
play_stat['ATFormPts'] = play_stat['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
play_stat['HTWinStreak3'] = play_stat['HTFormPtsStr'].apply(get_3game_ws)
play_stat['HTWinStreak5'] = play_stat['HTFormPtsStr'].apply(get_5game_ws)
play_stat['HTLossStreak3'] = play_stat['HTFormPtsStr'].apply(get_3game_ls)
play_stat['HTLossStreak5'] = play_stat['HTFormPtsStr'].apply(get_5game_ls)

play_stat['ATWinStreak3'] = play_stat['ATFormPtsStr'].apply(get_3game_ws)
play_stat['ATWinStreak5'] = play_stat['ATFormPtsStr'].apply(get_5game_ws)
play_stat['ATLossStreak3'] = play_stat['ATFormPtsStr'].apply(get_3game_ls)
play_stat['ATLossStreak5'] = play_stat['ATFormPtsStr'].apply(get_5game_ls)

play_stat.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [12]:
# Get Goal Difference
play_stat['HTGD'] = play_stat['HTGS'] - play_stat['HTGC']
play_stat['ATGD'] = play_stat['ATGS'] - play_stat['ATGC']

# Diff in points
play_stat['DiffPts'] = play_stat['HTP'] - play_stat['ATP']
play_stat['DiffFormPts'] = play_stat['HTFormPts'] - play_stat['ATFormPts']

# Diff in last year positions
play_stat['DiffLP'] = play_stat['HomeTeamLP'] - play_stat['AwayTeamLP']

In [13]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
play_stat.MW = play_stat.MW.astype(float)

for col in cols:
    play_stat[col] = play_stat[col] / play_stat.MW

In [14]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
    
play_stat['FTR'] = play_stat.FTR.apply(only_hw)

# 2017-18 season
play_stat_test = play_stat[(len(play_stat[col])-380):]

In [15]:
play_stat.to_csv("final_dataset.csv")
play_stat_test.to_csv("test.csv")

## Notes for Football Data
### Key to results data:
Div = League Division

Date = Match Date (dd/mm/yy)

HomeTeam = Home Team

AwayTeam = Away Team

FTHG = Full Time Home Team Goals

FTAG = Full Time Away Team Goals

FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)

HTHG = Half Time Home Team Goals

HTAG = Half Time Away Team Goals

HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

### Match Statistics (where available)
Attendance = Crowd Attendance

Referee = Match Referee

HS = Home Team Shots

AS = Away Team Shots

HST = Home Team Shots on Target

AST = Away Team Shots on Target

HHW = Home Team Hit Woodwork

AHW = Away Team Hit Woodwork

HC = Home Team Corners

AC = Away Team Corners

HF = Home Team Fouls Committed

AF = Away Team Fouls Committed

HO = Home Team Offsides

AO = Away Team Offsides

HY = Home Team Yellow Cards

AY = Away Team Yellow Cards

HR = Home Team Red Cards

AR = Away Team Red Cards

HBP = Home Team Bookings Points (10 = yellow, 25 = red)

ABP = Away Team Bookings Points (10 = yellow, 25 = red)

### Key to 1X2 (match) betting odds data:

B365H = Bet365 home win odds

B365D = Bet365 draw odds

B365A = Bet365 away win odds

BSH = Blue Square home win odds

BSD = Blue Square draw odds

BSA = Blue Square away win odds

BWH = Bet&Win home win odds

BWD = Bet&Win draw odds

BWA = Bet&Win away win odds

GBH = Gamebookers home win odds

GBD = Gamebookers draw odds

GBA = Gamebookers away win odds

IWH = Interwetten home win odds

IWD = Interwetten draw odds

IWA = Interwetten away win odds

LBH = Ladbrokes home win odds

LBD = Ladbrokes draw odds

LBA = Ladbrokes away win odds

PSH = Pinnacle Sports home win odds

PSD = Pinnacle Sports draw odds

PSA = Pinnacle Sports away win odds

SOH = Sporting Odds home win odds

SOD = Sporting Odds draw odds

SOA = Sporting Odds away win odds

SBH = Sportingbet home win odds

SBD = Sportingbet draw odds

SBA = Sportingbet away win odds

SJH = Stan James home win odds

SJD = Stan James draw odds

SJA = Stan James away win odds

SYH = Stanleybet home win odds
SYD = Stanleybet draw odds

SYA = Stanleybet away win odds

VCH = VC Bet home win odds

VCD = VC Bet draw odds

VCA = VC Bet away win odds

WHH = William Hill home win odds

WHD = William Hill draw odds

WHA = William Hill away win odds


Bb1X2 = Number of BetBrain bookmakers used to calculate match odds averages and maximums

BbMxH = Betbrain maximum home win odds

BbAvH = Betbrain average home win odds

BbMxD = Betbrain maximum draw odds

BbAvD = Betbrain average draw win odds

BbMxA = Betbrain maximum away win odds

BbAvA = Betbrain average away win odds



### Key to total goals betting odds:

BbOU = Number of BetBrain bookmakers used to calculate over/under 2.5 goals (total goals) averages and maximums

BbMx>2.5 = Betbrain maximum over 2.5 goals

BbAv>2.5 = Betbrain average over 2.5 goals

BbMx<2.5 = Betbrain maximum under 2.5 goals

BbAv<2.5 = Betbrain average under 2.5 goals

GB>2.5 = Gamebookers over 2.5 goals

GB<2.5 = Gamebookers under 2.5 goals

B365>2.5 = Bet365 over 2.5 goals

B365<2.5 = Bet365 under 2.5 goals


### Key to Asian handicap betting odds:

BbAH = Number of BetBrain bookmakers used to Asian handicap averages and maximums

BbAHh = Betbrain size of handicap (home team)

BbMxAHH = Betbrain maximum Asian handicap home team odds

BbAvAHH = Betbrain average Asian handicap home team odds

BbMxAHA = Betbrain maximum Asian handicap away team odds

BbAvAHA = Betbrain average Asian handicap away team odds

GBAHH = Gamebookers Asian handicap home team odds

GBAHA = Gamebookers Asian handicap away team odds

GBAH = Gamebookers size of handicap (home team)

LBAHH = Ladbrokes Asian handicap home team odds

LBAHA = Ladbrokes Asian handicap away team odds

LBAH = Ladbrokes size of handicap (home team)

B365AHH = Bet365 Asian handicap home team odds

B365AHA = Bet365 Asian handicap away team odds

B365AH = Bet365 size of handicap (home team)